## EXTRACTING IMAGE FROM IMAGE , SEPARATING BACKGROUND AND NON IMPORTANT PARTS LIKE FACE, OTHER OBECTS IN THE PICTURE

In [ ]:
!pip install python-opencv

ERROR: Could not find a version that satisfies the requirement python-opencv (from versions: none)
ERROR: No matching distribution found for python-opencv


In [ ]:
from google.colab import drive
drive.mount('/gdrive',)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
%ls /gdrive/My\ Drive

 chromedriver
 code.zip
'Colab Notebooks'/
 Cut/
 data/
'Data Sheets'/
 deep-learning-face-detection/
 dependencies/
 detection_temp.zip
 downloads/
 foo.txt
 instance_segmentation/
 keras-multi-label/
 model2data/
 new_bin.pickle
 new_fashion.model
 plot_trend.png
 temp/
 temp12877wIQuhRretJCUlIXAn-ITTC5M6V7a.jpg
 temp12bCp4qMDQheJNmv-LpuqXwyAsltaKRbR.jpg
 temp12BOTqML_jVCWMcO5qIsshPjw8pBMwnrC.jpg
 temp12R4eAddEHw-JW8nENx7qWRcwOu6e1sRs.jpg
 temp12Rl_G7MSdUcKOJSSrB1iHJtAp26Ln79a.jpg
 temp132j0bW5oUHm8uqJlS3EXuLycT7G2rqYI.jpg
 temp13GvGhvRFLQHN_fP4XjjCmQwLrZimyJBJ.jpg
 temp142NZUQmoTg9pBrCEkbiNmv5aClIkVWeD.jpg
 temp14A88rXnPhAk7D1yTpEnR6rWDof6qzkHo.jpg
 temp14_eMsXRTR2o9RJ1UjdnijmzdRaTvU2XR.jpg
 temp14joFGPRweZ9pGxMS9IexkXI-G1Z2RClb.jpg
 temp14K_1XMsfHCrlcCbA6Gof0zhjynXqIS8R.jpg
 temp14N6wnPOhiSe8bZUaKVe5Xnc6LBet_z59.jpg
 temp1Vy92UAWClXE1INbT6ZOEO1EGq6m8-NQi.jpg
 temp1W3bw3cyG88-m8Otnx98COF2cwGx4Iult.jpg
 temp1Y_cLO44EjA1mTw8dN_tOoBXLkFez933u.jpg
 temp1YdKSO312-v1LFZfyalsk57awZGHLRaXm.

In [ ]:
%cd /gdrive/My\ Drive

/gdrive/My Drive


In [ ]:
from PIL import Image
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import numpy as np
import cv2
import os
import sys
import io


In [ ]:
# define the path to the face detector
FACE_DETECTOR_PATH = "dependencies/haarcascade_frontalface_default.xml"

path = "google/yellow_dress/"

OUT_PATH = "Cut/" + path
DOWNLOAD_PATH = "data/" + path

In [ ]:
def facedetection(image):
    # convert the image to grayscale, load the face cascade detector and detect faces in the image

    try:
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        print("Image already grey")
    detector = cv2.CascadeClassifier(FACE_DETECTOR_PATH)
    drects = detector.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=5,
                                      minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # construct a list of bounding boxes from the detection
    drects = [(int(x), int(y), int(x + w), int(y + h)) for (x, y, w, h) in drects]
    
    # Ignore false faces since multiple faces are detected on certain images
    height,width = img_gray.shape
    drects.sort(key=lambda x:x[1],reverse=False)
    if len(drects)>0:
       rects = drects[:1]
       lx,ly,rx,ry = rects[0]
       if ry>height/2 or np.var( img_gray[ly:ry,lx:rx])<1:
          rects = [] 
    else:
       rects = []
 
    return rects


In [ ]:
def findbound(img):
    height,width = img.shape

    # Find the rectangle boundary in four directions
    for i in range(width):
        if img[:,i].tolist().count(255) > 2:
           left = i
           break

    for i in range(width-1,-1,-1):
        if img[:,i].tolist().count(255) > 2:
           right = i
           break

    for i in range(height):
        if img[i,:].tolist().count(255) > 2:
           top = i
           break

    for i in range(height-1,-1,-1):
        if img[i,:].tolist().count(255) > 2:
           bottom = i
           break

    bound = [(top,bottom,left,right)]
    return bound


In [ ]:
def filterboundarea(img, bvalue, svalue, rect):
    top,bottom,left,right = rect

    mask = np.zeros(img.shape[:2],np.uint8)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    for (y,x), value in np.ndenumerate(img_gray):
        if y>top and y<bottom and x>left and x<right:
           if img_gray[y,x] > bvalue+20 or img_gray[y,x]<bvalue-20:
              mask[y,x] = 1

    if not np.isnan(svalue):
       for (y,x), value in np.ndenumerate(img_gray):
        if y>top and y<bottom and x>left and x<right:
           if img_gray[y,x] < svalue+50 and img_gray[y,x]>svalue-50:
              mask[y,x] = 0

    fimg = img*mask[:,:,np.newaxis]

    return fimg


In [ ]:
def backgroundmodel(img,rect):
    top,bottom,left,right = rect
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    mask = np.ones(img.shape[:2],np.uint8)
    mask[top:bottom,left:right]=0
    img_gray = img_gray*mask
    bvalue = np.median(img_gray[img_gray>0])

    return bvalue


In [ ]:
def skinmodel(img,rect):
    startX,startY,endX,endY = rect
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    crop_img = img_gray[startY:endY, startX:endX]
    
    hist = [0]*256 
    for (y,x),value in np.ndenumerate(crop_img):
        hist[crop_img[y,x]] += 1 

    return hist.index(max(hist))


In [ ]:
def removebackground(image,rects):
    # select left corner 15x15 patch
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)
    height,width = img_gray.shape

    v = np.average(img_gray[1:16])
    sigma = 0.33
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(img_blur, lower, upper)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilated = cv2.dilate(edged, kernel, iterations = 3)
    
    # Find the bound area
    boundrect = findbound(dilated)

    # Get the background RGB model
    bvalue = backgroundmodel(image, boundrect[0])
    
    # Get the skin model
    if len(rects)>0:
       svalue = skinmodel(image,rects[0])
    else:
       svalue = np.nan

    # Exclude face area
    if len(rects)>0:
        lx,ly,rx,ry = rects[0]
        top,bottom,left,right = boundrect[0]
        boundrect=[]
        boundrect = [(ry,bottom,left,right)]

    # Filter out background in bounded area
    rgimg = filterboundarea(image, bvalue, svalue, boundrect[0])
    return rgimg 


In [ ]:
def extractobj(filepath):
    try:
        image = cv2.imread(filepath)
        filename = filepath.split('/')[-1] 
        dotidx = filename.index('.') 
        new_filename = OUT_PATH + filename[:dotidx] + "_cut.png" #+ filename[dotidx:]
        if(os.path.isfile(new_filename)):
          print(new_filename + " already exists... Skipping...")
          return
        
        print(new_filename)
    
        # face detection 
        rects = facedetection(image)

        # detect and remove background
        fimg = removebackground(image,rects)
    
        cv2.imwrite(new_filename,fimg)
    except:
        print("Failed for this object")
        filename = filepath.split('/')[-1] 
        dotidx = filename.index('.') 
        new_filename = OUT_PATH + filename[:dotidx] + "_cut" + filename[dotidx:]
        cv2.imwrite(new_filename,image)



In [ ]:
def multithreadtask(num_thread=1):
    # Get the file list in saved Google drive folder
    img_list = []
    for fimg in os.listdir(DOWNLOAD_PATH):
        if fimg.lower().endswith(('.jpg','.jpeg','.png')):
           extractobj(os.path.join(DOWNLOAD_PATH,fimg)) 
    
    #print("Total number:%d" % len(img_list))
    
    # Process the files in that folder in multi-threads 
    #pool = ThreadPool(num_thread)
    #pool.map(extractobj, img_list)
    
    #pool.close()
    #pool.join() 
    
    #return len(img_list) 


In [ ]:
multithreadtask()

Cut/google/yellow_dress/image1_cut.png
Cut/google/yellow_dress/image2_cut.png
Cut/google/yellow_dress/image3_cut.png
Cut/google/yellow_dress/image4_cut.png
Cut/google/yellow_dress/image5_cut.png
Cut/google/yellow_dress/image6_cut.png
Cut/google/yellow_dress/image7_cut.png
Cut/google/yellow_dress/image8_cut.png
Cut/google/yellow_dress/image9_cut.png
Cut/google/yellow_dress/image10_cut.png
Cut/google/yellow_dress/image11_cut.png
Cut/google/yellow_dress/image12_cut.png
Cut/google/yellow_dress/image13_cut.png
Cut/google/yellow_dress/image14_cut.png
Cut/google/yellow_dress/image15_cut.png
Cut/google/yellow_dress/image16_cut.png
Cut/google/yellow_dress/image17_cut.png
Cut/google/yellow_dress/image18_cut.png
Cut/google/yellow_dress/image19_cut.png
Cut/google/yellow_dress/image20_cut.png
Cut/google/yellow_dress/image21_cut.png
Cut/google/yellow_dress/image22_cut.png
Cut/google/yellow_dress/image23_cut.png
Cut/google/yellow_dress/image24_cut.png
Cut/google/yellow_dress/image25_cut.png
Cut/googl